# Import Libraries

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Preprocession Text
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing.text import Tokenizer
from IPython.display import clear_output
from imblearn.over_sampling import SMOTE
from nlp_id.lemmatizer import Lemmatizer
from nltk.tokenize import word_tokenize
from collections import Counter
from string import punctuation

# Report
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# Modelling
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

# Utility
from pandas import read_csv
from pandas import DataFrame
import pickle, re, json, time
import numpy as np
from requests import get
from io import StringIO
import nltk, re, json, csv

# Load Dataset

In [4]:
dataset_path = './pubg_dataset.csv'
review = read_csv(dataset_path)
print("\"{}\" is loaded succesfully".format(dataset_path))

# Create New Dataframe to keep every text preprocessing result
df = DataFrame(review['content'])

"./pubg_dataset.csv" is loaded succesfully


# Features Extraction

## Function Declaration

### Text Cleaning

In [50]:
# REMOVE SPECIAL CHARACTERS & CASE FOLDING
def cleaning(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', str(text))              # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text)                   # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text)                          # menghapus RT
    text = re.sub(r"http\S+", '', text)                         # menghapus link
    text = re.sub(r'[0-9]+', '', text)                          # menghapus angka
    text = re.sub(r"[,.;@#?!&$]+\ *", " ", text)                # mengganti tanda baca dengan spasi
    text = re.sub(r'[^\w\s]', '', text)                         # menghapus karakter selain huruf dan angka
    text = text.replace('\n', ' ')                              # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', punctuation))   # menghapus semua tanda baca
    text = text.strip(' ')                                      # menghapus karakter spasi dari kiri dan kanan teks
    text = re.sub(r' +', ' ', text)                             # Remove double space
    return text.lower()

In [57]:
print("Proses Cleaning     : START")
df['text_clean'] = df['content'].apply(cleaning)
print("Proses Cleaning     : DONE\n")

sample = df.sample(n=1, random_state=1)
print("Printing sample...")
print("Text Original : ", sample['content'].values[0])
print("Text Output   : ", sample['text_clean'].values[0])

Proses Cleaning     : START
Proses Cleaning     : DONE

Printing sample...
Text Original :  Pubg mobile sekarang sering nge bug mulai dari voice chat gak keluar suara (gak fungsi), Bug masuk ke permainan, sampai bug server tidak merespon... Ini mengganggu kami par pemain.. Tolong segera diperbaiki
Text Output   :  pubg mobile sekarang sering nge bug mulai dari voice chat gak keluar suara gak fungsi bug masuk ke permainan sampai bug server tidak merespon ini mengganggu kami par pemain tolong segera diperbaiki


### Fixing Slangwords

In [58]:
# REMOVE SLANG WORDS
def fixSlangwords(text):
    words = text.split()
    fixed_words = []
    with open('./resources/slangwords/slangwords_dict.json', mode='r') as file:
        slangwords = json.load(file)
        
    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word.lower())
    
    return ' '.join(fixed_words)

In [59]:
print("Proses Slangword    : START")
df['text_slangwords'] = df['text_clean'].apply(fixSlangwords)
print("Proses Slangword    : DONE\n")

sample = df.sample(n=1, random_state=1)
print("Printing sample...")
print("Text Original : ", sample['text_clean'].values[0])
print("Text Output   : ", sample['text_slangwords'].values[0])

Proses Slangword    : START
Proses Slangword    : DONE

Printing sample...
Text Original :  pubg mobile sekarang sering nge bug mulai dari voice chat gak keluar suara gak fungsi bug masuk ke permainan sampai bug server tidak merespon ini mengganggu kami par pemain tolong segera diperbaiki
Text Output   :  pubg mobile sekarang sering nge bug mulai dari voice chat tidak keluar suara tidak fungsi bug masuk ke permainan sampai bug server tidak merespon ini mengganggu kami par pemain tolong segera diperbaiki


### Remove Stopwords

In [63]:
# REMOVE STOPWORDS
def stopwordsRemove(text):
    words = text.split()
    fixed_words = []
    with open('./resources/stopwords/stopwords_dict.json', mode='r') as file:
        stopwords = json.load(file)
        
    for word in words:
        if word not in stopwords:
            fixed_words.append(word)

    return ' '.join(fixed_words)

In [64]:
print("Proses Stopword     : START")
df['text_stopwords'] = df['text_slangwords'].apply(stopwordsRemove)
print("Proses Stopword     : DONE\n")

sample = df.sample(n=1, random_state=1)
print("Printing sample...")
print("Text Original : ", sample['text_slangwords'].values[0])
print("Text Output   : ", sample['text_stopwords'].values[0])

Proses Stopword     : START
Proses Stopword     : DONE

Printing sample...
Text Original :  pubg mobile sekarang sering nge bug mulai dari voice chat tidak keluar suara tidak fungsi bug masuk ke permainan sampai bug server tidak merespon ini mengganggu kami par pemain tolong segera diperbaiki
Text Output   :  pubg mobile nge bug voice chat keluar suara fungsi bug masuk permainan bug server merespon mengganggu par pemain diperbaiki


### Lemmatizing Words

In [65]:
# LEMMATIZING
def lemmatizerWords(text):
    words = text.split()
    lemmatized_words = []
    lemmatizer  = Lemmatizer()
    lemmatized_words.append(lemmatizer.lemmatize(word.lower()) for word in words)
    return ' '.join(lemmatized_words[0])

In [66]:
text = "Menggunakan akal itu sia-sia"
lemmatizerWords(text)

'guna akal itu sia'

In [67]:
print("Proses Lemmatizing  : START")
df['text_lemmatize'] = df['text_stopwords'].apply(lemmatizerWords)
print("Proses Lemmatizing  : DONE\n")

sample = df.sample(n=1, random_state=1)
print("Printing sample...")
print("Text Original : ", sample['text_stopwords'].values[0])
print("Text Output   : ", sample['text_lemmatize'].values[0])

Proses Lemmatizing  : START
Proses Lemmatizing  : DONE

Printing sample...
Text Original :  pubg mobile nge bug voice chat keluar suara fungsi bug masuk permainan bug server merespon mengganggu par pemain diperbaiki
Text Output   :  pubg mobile nge bug voice chat keluar suara fungsi bug masuk main bug server merespon ganggu par main baik


## Save Dataset

In [69]:
df.to_csv('./pubg_preprocessing.csv', index=False)

## Load Dataset preprocessing

In [2]:
df = read_csv('./pubg_preprocessing.csv')
df.head()

,content,text_clean,text_slangwords,text_stopwords,text_lemmatize
0,"Tencent, saya bukan player pro,. Tapi kami sec...",tencent saya bukan player pro tapi kami secara...,tencent saya bukan player profesional tapi kam...,tencent player profesional mengeluh updatean b...,tencent player profesional keluh updatean bug ...
1,Bug setelah update. 1 . Turun pesawat ga kebag...,bug setelah update turun pesawat ga kebagian p...,bug setelah update turun pesawat tidak kebagia...,bug update turun pesawat kebagian parasut kara...,bug update turun pesawat bagi parasut karakter...
2,"Keseluruhan game nya udah bagus, apalagi setel...",keseluruhan game nya udah bagus apalagi setela...,keseluruhan game nya sudah bagus apalagi setel...,game nya bagus update graphics makinn mantap k...,game nya bagus update graphics makinn mantap k...
3,"Pubg setelah update bnyak aja masalahnya, sepe...",pubg setelah update bnyak aja masalahnya seper...,pubg setelah update bnyak saja masalahnya sepe...,pubg update bnyak masalahnya bug map pas game ...,pubg update bnyak masalah bug map pas game pla...
4,Game bagus...tapi masih banyak bug...masih per...,game bagus tapi masih banyak bug masih perlu d...,game bagus tapi masih banyak bug masih perlu d...,game bagus bug diperbaiki dalam mode tdm bug p...,game bagus bug baik dalam mode tdm bug player ...


## Create Words Dictionary

In [3]:
# Counter words function declaration
def counter_word(texts):
    count = Counter()
    for text in texts.values:
        for word in text.split():
            count[word] += 1
    return count

# Count unique words
counter = counter_word(df['text_lemmatize'])
num_unique_words = len(counter)     # 20993
# counter.most_common(5)
print("\nUnique words : ", num_unique_words)

# Tokenizer word from text_lemmatize coloumn
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(df['text_lemmatize'])

# Get Word Index from tokenizer
word_index = tokenizer.word_index

# Save word dictionary to csv
with open('./word_dictionary.json', mode='w') as file:
  json.dump(word_index, file, indent=4)


Unique words :  19967


In [37]:
print(word_index)

{'game': 1, 'nya': 2, 'bug': 3, 'update': 4, 'baik': 5, 'pubg': 6, 'main': 7, 'bagus': 8, 'hp': 9, 'tencent': 10, 'player': 11, 'kasih': 12, 'musuh': 13, 'jaring': 14, 'masuk': 15, 'mohon': 16, 'ram': 17, 'pas': 18, 'lag': 19, 'iya': 20, 'pakai': 21, 'parah': 22, 'suka': 23, 'banget': 24, 'turun': 25, 'gb': 26, 'baru': 27, 'dalam': 28, 'sinyal': 29, 'kecewa': 30, 'terima': 31, 'lancar': 32, 'grafik': 33, 'download': 34, 'bintang': 35, 'mode': 36, 'nyaman': 37, 'gamenya': 38, 'teman': 39, 'guna': 40, 'senjata': 41, 'pesawat': 42, 'season': 43, 'keluar': 44, 'event': 45, 'mati': 46, 'kadang': 47, 'mobile': 48, 'map': 49, 'hilang': 50, 'server': 51, 'data': 52, 'skin': 53, 'patah': 54, 'cepat': 55, 'sih': 56, 'developer': 57, 'kan': 58, 'login': 59, 'wifi': 60, 'habis': 61, 'terimakasih': 62, 'stabil': 63, 'bikin': 64, 'frame': 65, 'buka': 66, 'susah': 67, 'buat': 68, 'biar': 69, 'coba': 70, 'kurang': 71, 'fps': 72, 'ulang': 73, 'seru': 74, 'koneksi': 75, 'ping': 76, 'kayak': 77, 'device'

In [64]:
word_df = DataFrame(word_index.items(), columns=["word", "id"])
word_df.head()
word_df.to_csv('./word_dictionary.csv', index=False)

In [67]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_indonesia = set(stopwords.words('indonesian'))

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopwords_sastrawi = factory.get_stop_words()

[nltk_data] Downloading package stopwords to /home/yelf/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [69]:
def stopwordNLTK(text):
    words = text.split()
    return " ".join([word for word in words if word.lower() not in stopwords_indonesia])

def stopwordSastrawi(text):
    words = text.split()
    return " ".join([word for word in words if word.lower() not in stopwords_sastrawi])

In [70]:
df['text_stopwordsNLTK'] = df['text_lemmatize'].apply(stopwordNLTK)
df['text+stopwordsSastrawi'] = df['text_stopwordsNLTK'].apply(stopwordSastrawi)

In [72]:
# Counter words function declaration
def counter_word(texts):
    count = Counter()
    for text in texts.values:
        for word in text.split():
            count[word] += 1
    return count

# Count unique words
counter = counter_word(df['text_stopwordsNLTK'])
num_unique_words = len(counter)     # 20993
# counter.most_common(5)
print("\nUnique words : ", num_unique_words)

# Tokenizer word from text_lemmatize coloumn
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(df['text_stopwordsNLTK'])

# Get Word Index from tokenizer
word_index = tokenizer.word_index

# Save word dictionary to csv
with open('./word_dictionary.json', mode='w') as file:
  json.dump(word_index, file, indent=4)


Unique words :  19760
